In [2]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

from sklearn.datasets import make_classification
from sklearn.cluster import KMeans,MeanShift,MiniBatchKMeans
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,Normalizer
from sklearn.compose import ColumnTransformer

import category_encoders as ce

In [4]:
#read in data
customers = pd.read_csv('./data/customers.csv')
customers.head()

,customer_ID,sex,registration_date,first_rent_date,customer_age
0,1,female,01.09.20 00:14,01.09.20 01:47,34
1,2,male,31.08.20 21:13,31.08.20 23:50,28
2,3,male,31.08.20 20:53,31.08.20 23:06,23
3,4,female,31.08.20 20:50,31.08.20 23:45,24
4,5,female,31.08.20 20:45,31.08.20 23:33,28


In [5]:
rentals = pd.read_csv('./data/rentals.csv')
rentals.head()

,place,package,rent_id,customer_ID,rental_start_date,distance,duration,price
0,Paris,minute_based_rental,d9b784a3-4,17091,15.04.20,2,7,2.07
1,Madrid,minute_based_rental,0906b4a9-e,66145,15.02.20,4,12,2.38
2,Madrid,minute_based_rental,43d2d8c0-6,62554,24.07.20,4,15,4.10
3,Paris,minute_based_rental,b92469d5-0,31508,13.01.20,10,27,5.40
4,Madrid,minute_based_rental,b3c72862-3,66064,08.06.20,2,8,1.79
